In [1]:
# Dependencies
from matplotlib import pyplot as plt
from scipy.stats import linregress
import numpy as np
from sklearn import datasets 
import pandas as pd
import time
import requests
from census import Census
#pd.set_option('display.max_rows', 30000)
# Census API Key
from config import census_key
c = Census(census_key, year=2015)

### Retrieve Crime Data from FBI Crime Database - UCR

In [2]:
#Store file path for crime data in a variable
crime_data = "../Data/Offenses_Known_to_Law_Enforcement_by_State_by_City_2015.xls"

#Read data into dataframe
crime_data_df=pd.read_excel(crime_data)

#Extract the state into a list so all rows in "state" column can have a state value
state = crime_data_df["State"]
states=[]
for s in state:
    if pd.isnull(s):
        states.append(current_state)
    else:
        current_state = s
        states.append(current_state)
        
#Add newly created list into the dataframe and remove the old column
crime_data_df["NEW_STATE"] = states
clean_crime_data_df=crime_data_df[["NEW_STATE", "City", "Population", "Violent Crime"]] 
clean_crime_data_df=clean_crime_data_df.rename(columns={"NEW_STATE":"State"})

#Remove any trailing numbers at the end of state and city names
clean_crime_data_df["State"] = clean_crime_data_df["State"].map(lambda x: x.rstrip("1234567890"))
clean_crime_data_df["City"] = clean_crime_data_df["City"].map(lambda x: x.rstrip("1234567890"))

#Calculate the rate of violent crime per capita
clean_crime_data_df["Violent Crime Rate"] = (clean_crime_data_df["Violent Crime"]/clean_crime_data_df["Population"])*100

#Sort out the two cities in each state
